In [1]:
import pandas as pd 
import numpy as np
import datetime
import time
import requests

# 上市櫃外資投信買賣超資料

## 爬蟲程式

In [2]:
#主程式
def crawl_foreign(date):
    
    datestr = date.strftime('%Y%m%d')
    try:
        res = f"https://www.twse.com.tw/fund/TWT38U?response=html&date={datestr}"
        rawdf = pd.read_html(res)[0]
    
    except Exception as e:
        print(f'{date} cannot get data')
        return None

    df = rawdf[rawdf.columns[1:6]]
    df.columns = df.columns.get_level_values(2)
    df = df.rename(columns = {'證券代號':'stock_id','證券名稱':'stock_name',"買進股數":"外資買進",'賣出股數':'外資賣出', '買賣超股數':'外資買賣超' })
    df['stock_id'] = df['stock_id'].astype(str)
    df = df[df.stock_id.str.len()==4]
    df.insert(0,'date',date)
    df = df.set_index(['stock_id','stock_name','date'])
    df.iloc[:,:] = (df.iloc[:,:]/1000).astype(int)
    return df


def crawl_sitca(date):
    datestr = date.strftime('%Y%m%d')
    try:
        res = f"https://www.twse.com.tw/fund/TWT44U?response=html&date={datestr}"
        rawdf = pd.read_html(res)[0]
    
    except:
        return None
    #取出要的資料並轉換column
    df = rawdf[rawdf.columns[1:6]]
    df.columns = df.columns.get_level_values(1)
    
    #column 重新命名並將stock_id轉換成文字檔
    df = df.rename(columns = {'證券代號':'stock_id','證券名稱':'stock_name', "買進股數":"投信買進",'賣出股數':'投信賣出', '買賣超股數':'投信買賣超' })
    df['stock_id'] = df['stock_id'].astype(str)
    
    #把權證等資料剔除，只留下股票
    df = df[df.stock_id.str.len()==4]
    
    #加入日期，再設定index
    df.insert(0,'date',date)
    df = df.set_index(['stock_id','stock_name', 'date'])
    
    #因為上市資料是一股，因此除以1000
    df.iloc[:,:] = (df.iloc[:,:]/1000).astype(int)
    
    return df

def crawl_foreign_counter(date):
    
    date_rc = f"{date.year-1911}/{str(date)[5:7]}/{str(date)[8:10]}"
    l = ['buy', 'sell']
    result = pd.DataFrame()
    try:
        for i in l:
            res = (f'https://www.tpex.org.tw/web/stock/3insti/qfii_trading/forgtr_result.php?l=zh-tw&t=D&type={i}&d={date_rc}&s=0,asc&o=htm')
            raw = pd.read_html(res)[0]
            if len(raw)==1:
                print(f'{date} cannot get data')
            else:
                df = raw[raw.columns[1:6]]
                df.columns = df.columns.get_level_values(2)
                df = df.rename(columns = {'代號':'stock_id','名稱':'stock_name',"買進":"外資買進",'賣出':'外資賣出', '買賣超(仟股)':'外資買賣超' })
                df = df.drop(df.tail(1).index)
                df['stock_id'] = df['stock_id'].astype(str)
                df = df[df.stock_id.str.len()==4]
                df.insert(0,'date', date)
                df = df.set_index(['stock_id', 'stock_name', 'date'])
                result = result.append(df)
        
    except Exception as e:
        #print(f'{date} cannot get data')
        return None
    
    result.iloc[:,:] = result.astype(int)
    return result   

def crawl_sitca_counter(date):
    
    date_rc = f"{date.year-1911}/{str(date)[5:7]}/{str(date)[8:10]}"
    l = ['buy', 'sell']
    result = pd.DataFrame()
    
    try:
        for i in l:
            res = (f'https://www.tpex.org.tw/web/stock/3insti/sitc_trading/sitctr_result.php?l=zh-tw&t=D&type={i}&d={date_rc}&o=htm')
            raw = pd.read_html(res)[0]
            if len(raw)==1:
                print(f'{date} cannot get data')
                return None
            else:
                df = raw[raw.columns[1:6]]
                df.columns = df.columns.get_level_values(1)
                df = df.rename(columns = {'代號':'stock_id','名稱':'stock_name',"買進":"投信買進",'賣出':'投信賣出', '買賣超(仟股)':'投信買賣超' })
                df = df.drop(df.tail(1).index)
                df['stock_id'] = df['stock_id'].astype(str)
                df = df[df.stock_id.str.len()==4]
                df.insert(0,'date', date)
                df = df.set_index(['stock_id', 'stock_name', 'date'])
                result = result.append(df)
        
    except Exception as e:
        #print(f'{date} cannot get data')
        print(e)
        return None
        
    result.iloc[:,:] = result.astype(int)
    return result
        
#合併上市櫃
def combine_data(date):
    try:
        com = pd.concat([crawl_foreign(date),crawl_sitca(date)],axis=1, sort=False)
        com_counter = pd.concat([crawl_foreign_counter(date), crawl_sitca_counter(date)], axis=1, sort=False)
        df = pd.concat([com, com_counter])
        return df
    
    except Exception as e:
        return None


# 依日期取得資料    
def get_data(start, end):
    result = pd.DataFrame()
    
    for d in range(int((end -start).days)+1):
        try:
            date = start+datetime.timedelta(d)
            result = result.append(combine_data(date), sort=False)
           #if result.empty:
           #    print(f'{date} cannot get data')
           #else :
           #    print(f'{date}: success')
            time.sleep(10)
        
        except Exception as e:
            print(e)
            
    return result

## 更新資料

In [3]:
import sqlite3
import os


def update(date):
    con = sqlite3.connect('mydata.db')
    df = pd.read_sql('SELECT * FROM institutional_investors', con, index_col=['stock_id', 'stock_name', 'date'])
    df = df.drop_duplicates()
    df = df.sort_index(level='date')
    #get 最後一天日期
    start = df.index[-1][2]
    start = datetime.datetime.strptime(start, "%Y-%m-%d %H:%M:%S").date()
    
    #end = datetime.date(date)
    
    dfnew = df.append(get_data(start, date))
    dfnew = dfnew.drop_duplicates()
    dfnew.sort_index(level='date',inplace=True)
    dfnew.to_csv('foreign_backup.csv')
    dfnew.to_sql('institutional_investors', con, if_exists='replace')
    print("Success")
    return df
    

#def add_sql(df)

## 執行更新程式

In [4]:
%%time
import time
date = datetime.date.today()
#time.sleep(3)
df = update(date)

Success
CPU times: user 6.15 s, sys: 1.87 s, total: 8.02 s
Wall time: 33.8 s


In [5]:
df

,,,外資買進,外資賣出,外資買賣超,投信買進,投信賣出,投信買賣超
stock_id,stock_name,date,,,,,,
0050,元大台灣50,2020-01-02 00:00:00,41.0,1878.0,-1837.0,NaN,NaN,NaN
0052,富邦科技,2020-01-02 00:00:00,5.0,2.0,3.0,NaN,NaN,NaN
0056,元大高股息,2020-01-02 00:00:00,11.0,30.0,-19.0,NaN,NaN,NaN
0061,元大寶滬深,2020-01-02 00:00:00,645.0,0.0,645.0,NaN,NaN,NaN
1101,台泥,2020-01-02 00:00:00,4375.0,6371.0,-1996.0,0.0,5.0,-5.0
...,...,...,...,...,...,...,...,...
9939,宏全,2021-09-07 00:00:00,131.0,180.0,-49.0,0.0,20.0,-20.0
9941,裕融,2021-09-07 00:00:00,36.0,158.0,-122.0,0.0,1.0,-1.0
9945,潤泰新,2021-09-07 00:00:00,2886.0,3464.0,-578.0,NaN,NaN,NaN


In [6]:
con = sqlite3.connect('mydata.db')
df = pd.read_sql('SELECT * FROM institutional_investors', con, index_col=['stock_id', 'stock_name', 'date'])
df.drop_duplicates()

,,,外資買進,外資賣出,外資買賣超,投信買進,投信賣出,投信買賣超
stock_id,stock_name,date,,,,,,
0050,元大台灣50,2021-08-31 00:00:00,10174.0,280.0,9894.0,189.0,0.0,189.0
0056,元大高股息,2021-08-31 00:00:00,979.0,1356.0,-376.0,0.0,1017.0,-1017.0
1101,台泥,2021-08-31 00:00:00,14107.0,9824.0,4282.0,2.0,0.0,2.0
1102,亞泥,2021-08-31 00:00:00,3955.0,4525.0,-569.0,NaN,NaN,NaN
1103,嘉泥,2021-08-31 00:00:00,701.0,378.0,323.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
9939,宏全,2021-08-30 00:00:00,89.0,310.0,-221.0,0.0,150.0,-150.0
9941,裕融,2021-08-30 00:00:00,112.0,161.0,-48.0,NaN,NaN,NaN
9945,潤泰新,2021-08-30 00:00:00,718.0,1135.0,-416.0,NaN,NaN,NaN


In [7]:
df = df.sort_index(level=2)

# 檢查資料是否更新至最新

In [8]:
#check whether sort in index
#try
import sqlite3
con = sqlite3.connect('mydata.db')
df = pd.read_sql('SELECT * FROM institutional_investors', con, index_col=['stock_id', 'stock_name', 'date'])

df = df.drop_duplicates()
df = df.sort_index(level='date')
#get 最後一天日期
start = df.index[-1][2]
start = datetime.datetime.strptime(start, "%Y-%m-%d %H:%M:%S").date()
start

datetime.date(2021, 9, 1)

# 開始對資料進行分析

### 取出當天資料

In [6]:
#str(datetime.date.today())
#df.index.get_level_values('date')[0]
#df[df.index.get_level_values('date')==str(datetime.datetime.today())]

today = df[df.index.get_level_values(2) == df.index.get_level_values(2)[-1]]

today.ffill(0).sort_values(by='外資買賣超', ascending=False).head(10)

NameError: name 'df' is not defined

In [7]:
con1 = today.外資買賣超>0
con2 = today.投信買賣超>0
today[con1 & con2].sort_values(by = '外資買賣超', ascending=False)

NameError: name 'today' is not defined

In [21]:
df.query('stock_name=="尖點"').tail(5)

外資買進    外資賣出   外資買賣超  投信買進  投信賣出  \
stock_id stock_name date                                                      
8021     尖點         2021-07-22 00:00:00  2043.0  3211.0 -1168.0   NaN   NaN   
                    2021-07-23 00:00:00  5754.0  2088.0  3666.0   NaN   NaN   
                    2021-07-26 00:00:00  2195.0  4011.0 -1816.0   NaN   NaN   
                    2021-07-27 00:00:00  1272.0  1528.0  -256.0   NaN   NaN   
                    2021-07-28 00:00:00  3928.0  1783.0  2145.0   NaN   NaN   

                                         投信買賣超  
stock_id stock_name date                        
8021     尖點         2021-07-22 00:00:00    NaN  
                    2021-07-23 00:00:00    NaN  
                    2021-07-26 00:00:00    NaN  
                    2021-07-27 00:00:00    NaN  
                    2021-07-28 00:00:00    NaN

In [13]:
date_array = df.index.get_level_values(2).drop_duplicates()

In [52]:
#df = pd.DataFrame(df.外資買賣超)
#df = df.pivot_table(values='外資買賣超', columns='stock_id', index= 'date' )
df.index = pd.DatetimeIndex(df.index)
df.ffill(0)

stock_id,0050,0051,0052,0055,0056,0061,1101,1102,1103,1104,...,9942,9943,9944,9945,9946,9949,9951,9955,9958,9962
date,,,,,,,,,,,,,,,,,,,,,
2020-01-02,-1837.0,NaN,3.0,NaN,-19.0,645.0,-1996.0,-1027.0,6.0,-178.0,...,-12.0,NaN,-158.0,-105.0,109.0,5.0,5.0,24.0,-215.0,NaN
2020-01-03,-893.0,NaN,NaN,NaN,NaN,289.0,-2029.0,-2285.0,459.0,-36.0,...,-5.0,7.0,NaN,-132.0,-112.0,NaN,-40.0,117.0,-227.0,NaN
2020-01-06,-3725.0,NaN,NaN,NaN,NaN,309.0,-1409.0,-503.0,614.0,-149.0,...,NaN,-5.0,0.0,-485.0,-33.0,NaN,-15.0,9.0,-283.0,NaN
2020-01-07,-759.0,NaN,NaN,NaN,NaN,NaN,1621.0,1003.0,-291.0,-118.0,...,NaN,1.0,NaN,-203.0,822.0,NaN,-67.0,25.0,-332.0,NaN
2020-01-08,-2216.0,NaN,NaN,NaN,-32.0,302.0,-1593.0,1993.0,-276.0,-275.0,...,NaN,NaN,NaN,-571.0,-176.0,NaN,-26.0,-63.0,-510.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-09,-2435.0,NaN,NaN,NaN,-2248.0,NaN,-8917.0,-5142.0,-1823.0,-381.0,...,NaN,NaN,NaN,-608.0,NaN,NaN,NaN,NaN,-669.0,NaN
2021-07-12,8987.0,NaN,NaN,NaN,1874.0,NaN,-3417.0,-1620.0,-562.0,34.0,...,499.0,NaN,NaN,964.0,NaN,NaN,NaN,NaN,10.0,NaN
2021-07-13,9526.0,NaN,297.0,NaN,-2773.0,NaN,-2400.0,-2278.0,-558.0,-330.0,...,NaN,NaN,NaN,-1011.0,NaN,NaN,-328.0,NaN,-137.0,NaN


# 存入資料庫(not finished)

In [39]:
def update_table():
    dfread = pd.read_sql('select * from institutional_investors', con )
    start = (dfread.iloc[-1].date)
    start = datetime.datetime.strptime(start, "%Y-%m-%d %H:%M:%S").date()
    end = datetime.date(2020,2,15)
    dfread.append(get_data(start, end))
    
#datetime.datetime((dfread.iloc[-1].date))
#pd.to_datetime(dfread.iloc[-1].date)

In [2]:

#dfnew = dfnew.drop_duplicates(['stock_id', 'stock_name', 'date'], keep='last')

##start = datetime.date(2020,1,1)
#start = ()
#today = datetime.date(2020,2,1)
#
#
#
#end = datetime.date(2020,2,15)
#dfnew = get_data(start, end)
##start = start.to_datetime64()
##start = datetime.datetime.fromtimestamp()
##start.fromtimestamp()
##start.date()
#
#df = get_data(start, today)
#df.to_sql('institutional_investors', con, if_exists = 'replace')

In [9]:
df.append(combine_data(datetime.date.today()))

2021-07-09 cannot get data


,,,外資買進,外資賣出,外資買賣超,投信買進,投信賣出,投信買賣超
stock_id,stock_name,date,,,,,,
0050,元大台灣50,2021-07-07,2634.0,2044.0,589.0,NaN,NaN,NaN
0051,元大中型100,2021-07-07,6.0,3.0,3.0,NaN,NaN,NaN
0052,富邦科技,2021-07-07,26.0,12.0,14.0,NaN,NaN,NaN
0053,元大電子,2021-07-07,1.0,0.0,1.0,NaN,NaN,NaN
0055,元大MSCI金融,2021-07-07,0.0,3.0,-3.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
8937,合騏,2021-07-08,1.0,0.0,1.0,NaN,NaN,NaN
8938,明安,2021-07-08,276.0,799.0,-523.0,NaN,NaN,NaN
8942,森鉅,2021-07-08,461.0,138.0,323.0,NaN,NaN,NaN


# learning

In [80]:
# pandas中格子的字數
df[df.stock_id.str.len()==4]

,stock_id,stock_name,外資買進,外資賣出,外資買賣超
0,5351,鈺創,11714,8401,3313
1,6016,康和證,5753,2590,3163
2,6026,福邦證,3633,1272,2361
3,6015,宏遠證,4148,2062,2086
4,3260,威剛,8984,7084,1900
...,...,...,...,...,...
198,6199,天品,1,0,1
199,6486,互動,4,3,1
204,8077,洛碁,1,0,1
205,8176,智捷,0,0,0


In [13]:
df = pd.DataFrame([[1000,20000,30000],[30000,40000,50000],[60000,80000,90000]], index=['A','B','C'], columns=['a','b','c'])
df.iloc[:,1] = (df.iloc[:,1]/1000).astype(int)


In [14]:
df

,a,b,c
A,1000,20,30000
B,30000,40,50000
C,60000,80,90000
